In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse
from torch.utils.data.sampler import SubsetRandomSampler
from models import *
from utils import progress_bar
from tqdm import tqdm

In [4]:
parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#change the below default value  to tune hyper parameters
parser.add_argument('--lr', default= .01, type=float, help='learning rate')
parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
args = parser.parse_args(args=[])


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_val = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_val)


num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(0.2 * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)



trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=2, sampler= train_sampler)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=128,  num_workers=2, sampler=valid_sampler)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = VGG('VGG13')
#net = LeNet()
#net = ResNet18()
#net = PreActResNet18()
#net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
#net = SENet18()
# net = ShuffleNetV2(1)
net = EfficientNetB0()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt_tqdm.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)



# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
#testing
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            #print('entering')
            progress_bar(batch_idx, len(testloader))
            

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_tqdm.pth')
        best_acc = acc
        print('testing Accuracy: ', best_acc)

#validation
def validation(epoch):
    best_acc_val = 0
    net.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            #print('entering')
            progress_bar(batch_idx, len(valloader))
            
    # Save checkpoint.
    acc_val = 100.*correct/total
    if acc_val > best_acc_val :
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc_val,
            'epoch': epoch,
        }
        best_acc_val = acc_val
        print('validation Accuracy: ', best_acc_val)

for epoch in tqdm(range(start_epoch, start_epoch+5)):
    train(epoch)
    validation(epoch)
    test(epoch)


==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


  0%|          | 0/5 [00:00<?, ?it/s]


Epoch: 0
 [================================================================>]  Step: 37ms | Tot: 16s565ms | Loss: 1.751 | Acc: 37.530% (15012/4000 313/313 ===========================>...................................]  Step: 54ms | Tot: 7s413ms | Loss: 1.923 | Acc: 31.189% (5629/1804 141/313 
 [================================================================>]  Step: 16ms | Tot: 2s505 79/79 
Saving..
validation Accuracy:  44.05
 [================================================================>]  Step: 15ms | Tot: 2s164 100/100 


 20%|██        | 1/5 [00:39<02:36, 39.09s/it]

Saving..
testing Accuracy:  47.45

Epoch: 1
 [================================================================>]  Step: 35ms | Tot: 18s591ms | Loss: 1.464 | Acc: 48.545% (19418/4000 313/313  ==========================================>......................]  Step: 53ms | Tot: 12s561ms | Loss: 1.515 | Acc: 46.894% (12485/2662 208/313 
 [================================================================>]  Step: 14ms | Tot: 2s251 79/79 ==============================>.................................]  Step: 30ms | Tot: 1s79 39/79 =============================================>...................]  Step: 21ms | Tot: 1s591 56/79 
Saving..
validation Accuracy:  49.92
 [================================================================>]  Step: 14ms | Tot: 2s55 100/100 


...

Saving..
testing Accuracy:  59.27

Epoch: 3
 [================================================================>]  Step: 45ms | Tot: 16s865ms | Loss: 1.136 | Acc: 59.688% (23875/4000 313/313 =======>.....................................................]  Step: 52ms | Tot: 3s113ms | Loss: 1.182 | Acc: 58.347% (4257/729 57/313 ===========================>.....................................]  Step: 50ms | Tot: 7s339ms | Loss: 1.157 | Acc: 58.687% (10066/1715 134/313 ======================================================>..........]  Step: 52ms | Tot: 14s338ms | Loss: 1.146 | Acc: 59.228% (20090/3392 265/313 
 [================================================================>]  Step: 18ms | Tot: 2s236 79/79 ========================>......................................]  Step: 37ms | Tot: 956 33/79 
Saving..
validation Accuracy:  60.15
 [================================================================>]  Step: 17ms | Tot: 2s34 100/100 ====================>................................

 80%|████████  | 4/5 [02:36<00:38, 38.90s/it]

Saving..
testing Accuracy:  62.43

Epoch: 4
 [================================================================>]  Step: 36ms | Tot: 16s732ms | Loss: 1.043 | Acc: 62.883% (25153/4000 313/313 
 [================================================================>]  Step: 15ms | Tot: 2s187 79/79 
Saving..
validation Accuracy:  64.27
 [================================================================>]  Step: 18ms | Tot: 2s238 100/100 


100%|██████████| 5/5 [03:14<00:00, 38.99s/it]

Saving..
testing Accuracy:  66.05
